In [1]:
import os
os.environ['NUMBA_WARNINGS'] = '1'
import numpy as np
import numba

import matplotlib
%matplotlib notebook
import matplotlib.pyplot as plt

from matrixmethod import reflec_and_trans
import matrixmethod_numba
from matrixmethod_numba import reflec_and_trans as reflec_and_trans_numba, fields, fields_positions

/home/mpfluege/.local/lib/python3.4/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


In [2]:
from Anton_XRR import reflectivity
def anton_xrr(ang_deg, wl, n, rough):
    l = len(ang_deg)
    AOI = 90.-ang_deg
    wavelengths = np.array([wl]*l)
    n = np.outer(n, np.ones(l))
    t = np.concatenate([thick, [0]])
    t = np.outer(t, np.ones(l))
    sigma = rough
    kz_AOI, kx_AOI = reflectivity.helper.k_z_generator(AOI, wavelengths, n)
    r, trans = reflectivity.matrixmethod_xrr.amplitudes(n, wavelengths, kz_AOI, t, sigma)
    return r, trans

In [3]:
#n = np.array([1] + [1-1e-5+1e-6j]*20 + [1-2e-5+2e-6j] + [1]*11)
n = np.array([1] + [1-1e-5+1e-6j]*20 + [1-2e-5+2e-6j])
#rough = np.array([0.5]*22+[0]*10)
rough = np.array([0.]*21)
#thick = np.array([1.]*(21+10))
thick = np.array([1.]*20)
wl = 0.15
ang_deg = np.linspace(0, 1, 101)[1:]
ang = np.deg2rad(ang_deg)
#positions = np.linspace(2,-100, 102*10 +1)
positions = np.linspace(100, -120, 220*10+1)

In [4]:
mr, mt = reflec_and_trans(n, wl, ang, thick, rough)
nr, nt = fields(n, wl, ang, thick, rough)
nr = np.array(nr)
nt = np.array(nt)
fr, ft, posr, post = fields_positions(n, wl, ang, thick, rough, positions)
fr = np.array(fr)
ft = np.array(ft)
posr = np.array(posr)
post = np.array(post)
ar, at = anton_xrr(ang_deg, wl, n, rough)

In [6]:
#%timeit reflec_and_trans(n, wl, ang, thick, rough)
%timeit reflec_and_trans_numba(n, wl, ang, thick, rough)
%timeit fields(n, wl, ang, thick, rough)
%timeit fields_positions(n, wl, ang, thick, rough, positions)
%timeit anton_xrr(ang_deg, wl, n, rough)

1000 loops, best of 3: 426 µs per loop
1000 loops, best of 3: 599 µs per loop
10 loops, best of 3: 23.4 ms per loop
1000 loops, best of 3: 1.05 ms per loop


In [7]:
45.5/23.2

1.9612068965517242

In [ ]:
plt.figure()
plt.plot(ang_deg, np.abs(mr)**2, label='simple r')
#plt.plot(ang_deg, np.abs(np.array(Ap).T[0])**2, label='numba A+')
plt.plot(ang_deg, np.abs(ar[0])**2, label='anton r')
plt.plot(ang_deg, np.abs(nr[:,0])**2, label='numba r')
plt.plot(ang_deg, np.abs(fr[:,0])**2, label='fields r')
plt.legend(loc='lower right')
plt.yscale('log')
plt.figure()
#plt.plot(ang_deg, np.abs(np.array(Am).T[-1])**2, label='numba A-')
plt.plot(ang_deg, np.abs(at[-1])**2, label='anton t')
plt.plot(ang_deg, np.abs(mt)**2, label='simple t')
plt.plot(ang_deg, np.abs(nt[:,-1])**2, label='numba t')
plt.plot(ang_deg, np.abs(ft[:,-1])**2, label='fields t')
plt.yscale('log')
plt.legend(loc='lower right')

In [ ]:
Z = np.array([0]+list(-np.cumsum(thick)) + [-np.sum(thick)])
plt.figure()
#plt.plot(positions, np.abs(post[60]), label='Abs trans')
#plt.plot(positions, np.abs(posr[60]), label='Abs reflc')
#plt.plot(positions, np.real(post[60]), label='Re trans')
#plt.plot(positions, np.real(posr[60]), label='Re reflc')
plt.plot(positions, np.abs(post[60] + posr[60]), label='Abs total')
#plt.plot(positions, np.abs(post[60] + posr[60])**2, label='Abs total²')
plt.plot(positions, np.real(post[60] + posr[60]), label='Re total')
plt.plot(positions, np.abs(post[70] + posr[70]), label='Abs total 7')
#plt.plot(positions, np.abs(post[70] + posr[70])**2, label='Abs total² 7')
plt.plot(positions, np.real(post[70] + posr[70]), label='Re total 7')
#plt.plot(positions, np.imag(post[60] + posr[60]), label='Im total')
#plt.plot(positions, np.abs(posr[60]+post[60])**2, label='Abs total')
#plt.plot(Z, np.abs(fr[60]+ft[60,-1])**2, label='Abs total Z')
#plt.plot(positions, np.real(posr[0]), label='Re reflc')
#plt.plot(positions, np.real(posr[0]+post[0]), label='Re total')
#plt.plot(positions, np.imag(posr[0]+post[0]), label='Im total')
#plt.plot(positions, np.abs(posr[0]+post[0]), label='Abs total')
#plt.plot(positions, np.abs(posr[0])**2, label='reflc')
plt.legend(loc='upper left')
plt.grid()
#plt.yscale('log')

In [ ]:
from ipywidgets import interact

In [ ]:
@interact
def plot_int(x=60):
    fig, ax = plt.subplots()
    ax.plot(positions, np.abs(post[x] + posr[x]), label='Abs total')
    ax.plot(positions, np.real(post[x] + posr[x]), label='Re total')
    ax.legend(loc='upper left')
    ax.grid()


In [ ]:
nt[:,0]

In [ ]:
Aps = np.array(Ap).T
Aps.shape
Ams = np.array(Am).T
Ams.shape

In [ ]:
(rr - rrr[0]).max()

In [ ]:
rr

In [ ]:
np.abs(np.array(tt))**2

In [ ]:
np.abs(np.array(t))**2